#### Understanding Model management & Model registry

Before registring the model, look for below factors while deciding

* Model Run time
* Memory occupied (size of the model)
* Model metric
* computation engine (cpu or gpu)


In [1]:
import mlflow
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri("sqlite:///mlflow.db")

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [7]:
# using client to interact with mlflow 

client.get_experiment_by_name("nyc_taxi_experiment")

<Experiment: artifact_location=('/mnt/e/Machine_Learning_Projects/mlops_zoomcamp/02. Experiment '
 'Tracking/mlruns/1'), creation_time=1748346800773, experiment_id='1', last_update_time=1748346800773, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

In [4]:
# to create an experiment 

client.create_experiment("test_experiment")

'2'

In [8]:
# To get the best run from an experiment using below options which can be altered according to user needs

from mlflow.entities import ViewType 

runs = client.search_runs(
    experiment_ids="1",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: fcc2bf18b2fa4bc08373d785d468f483, rmse: 6.3609
run id: 7c921450488b458dbb56cc412da1d7a4, rmse: 6.3862
run id: 0e4d985c497f425693171efcb7d295f2, rmse: 6.4211
run id: 946a4ca7486c46c58d3d4f657badb516, rmse: 6.4681
run id: b3a609db575b4bf3936842da219f5811, rmse: 6.4697


In [16]:
import mlflow

# registering the Lasso reg model from other notebook
run_id = "504482ca54cd471ebf1aec6e3b5fcaf8"
model_uri = f"runs:/504482ca54cd471ebf1aec6e3b5fcaf8/ml_models"

mlflow.register_model(model_uri=model_uri, name="nyc_taxi_regressor")


Successfully registered model 'nyc_taxi_regressor'.
Created version '1' of model 'nyc_taxi_regressor'.


<ModelVersion: aliases=[], creation_timestamp=1748458687546, current_stage='None', description=None, last_updated_timestamp=1748458687546, name='nyc_taxi_regressor', run_id='504482ca54cd471ebf1aec6e3b5fcaf8', run_link=None, source=('/mnt/e/Machine_Learning_Projects/mlops_zoomcamp/02. Experiment '
 'Tracking/mlruns/1/504482ca54cd471ebf1aec6e3b5fcaf8/artifacts/ml_models'), status='READY', status_message=None, tags={}, user_id=None, version=1>

In [2]:
# getting the versions of registered model 

model_name = "nyc_taxi_regressor" 
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production


/tmp/ipykernel_21551/1267652280.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [3]:
# to transition the stage of any version of model 

model_version = 1
new_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=True
)

/tmp/ipykernel_21551/1182954759.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1748458687546, current_stage='Staging', description=None, last_updated_timestamp=1748551066046, name='nyc_taxi_regressor', run_id='504482ca54cd471ebf1aec6e3b5fcaf8', run_link=None, source=('/mnt/e/Machine_Learning_Projects/mlops_zoomcamp/02. Experiment '
 'Tracking/mlruns/1/504482ca54cd471ebf1aec6e3b5fcaf8/artifacts/ml_models'), status='READY', status_message=None, tags={}, user_id=None, version=1>

In [4]:
# if we need to update any of model version, we can use this function 

from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1748458687546, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2025-05-29', last_updated_timestamp=1748551698237, name='nyc_taxi_regressor', run_id='504482ca54cd471ebf1aec6e3b5fcaf8', run_link=None, source=('/mnt/e/Machine_Learning_Projects/mlops_zoomcamp/02. Experiment '
 'Tracking/mlruns/1/504482ca54cd471ebf1aec6e3b5fcaf8/artifacts/ml_models'), status='READY', status_message=None, tags={}, user_id=None, version=1>